Titanic project

3 fichiers csv : <li> gender_submission (exemple de BDD à soumettre) </li>
                 <li> - test (fichier d'entraînement/ BDD raw) </li>
                 <li> - train (aussi connu as "ground truth" fichier de vérification pour l'entraînement) </li>

Il va falloir d'abord analyser les données du fichier test.csv
                

1/ Analyse de données test.csv

On commence par importer les données

In [90]:
import pandas as  pd
import plotly.express as px
import numpy as np

# On import la data

df = pd.read_csv("../titanic-project/titanic_data/test.csv")
df


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


On regarde le type de données de chaque colonne

In [68]:
print(df.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


On regarde que la table s'organise en 12 colonnes : <ul>ID, passengerId, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin et Embarked</ul>
<ul>Variable / Definition / Key</ul>
<ul>PassengerID / ID table / Num(Int)</ul>
<ul>pclass /	Ticket class / Num(Int) 1,2 ou 3</ul>
<ul>sex /	Sex / Alpha 2 choices : male & female	</ul>
<ul>Age /	Age in years / Num(Decimal(10,1)) avec 0.5 de step</ul>
<ul>sibsp  /	# of siblings & spouses aboard the Titanic / Num(Int)</ul>
<ul>parch /	# of parents & children aboard the Titanic / Num(Int)</ul>
<ul>ticket	/ Ticket number / Alpha</ul>
<ul>fare /	Passenger fare / Num(Float)</ul>
<ul>cabin /	Cabin number / Alpha</ul>
<ul>embarked / Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton / Alpha 3 choices : C,Q,S</ul>

On vérifie maintenant que toutes les données dans chaque colonne correspond bien à leur Key définit plus haut précédemment

In [106]:
# On teste les valeurs dans Sexe pour voir si elles correspondent bien à la Key
df_test = df['Sex'].isin(['male','female'])
if df_test.all() == True:
    print('données vérifiées sexe')
else:
    print('non vérifiées')

# On teste les valeurs dans Pclass pour voir si elles correspondent bien à la Key
df_test = df['Pclass'].isin([1,2,3])
if df_test.all() == True:
    print('données vérifiées pclass')

# On teste les valeurs dans SibSp pour voir si elles correspondent bien à la Key
df_test = df['SibSp'] >= 0
if (df_test.all() == True) and (df_test.max() < 10):
    print('données vérifiées sibSp')

# On teste les valeurs dans Parch pour voir si elles correspondent bien à la Key
df_test = df['Parch'] >= 0
if (df_test.all() == True) and (df_test.max() < 10):
    print('données vérifiées Parch')

age_range =  df.loc[(df['Age'] > 0) & (df['Age']<100)]
df_test = df['Age'].isin([age_range])
print(df_test)





données vérifiées sexe
données vérifiées pclass
données vérifiées sibSp
données vérifiées Parch
0      False
1      False
2      False
3      False
4      False
       ...  
413    False
414    False
415    False
416    False
417    False
Name: Age, Length: 418, dtype: bool


On s'intéresse ensuite aux NaN. On vérifie s'il y a des données manquantes, et si oui, combien

In [10]:
print(df.isna().sum())

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


Age et Cabin ont des données manquantes. Ce sont tout les deux des types object, on fait donc un fillna("unknown")

In [ ]:
df.fillna("unknown")